In [1]:
import numpy as np

class Animal:
    
    # requried variables: needed for subclasses
    location = [[0], [0]]
    stepSize = 1
    mapSize = 0
    foodEaten = 0
    hunger = 0
    maxHunger = 0
    probRepro = 0
    sense = 1
    alive = True
    beStill = False
    
    def __init__(self, mapSize, location=None, maxHunger = 1000):
        
        if location == None:
            location = [np.random.randint(0, mapSize), np.random.randint(0, mapSize)]
        self.location = location
                
        self.mapSize = mapSize
        self.foodEaten = 0
        self.hunger = 0
        self.alive = True
        self.maxHunger = maxHunger
        
    def step(self):
        self.hunger = self.hunger + 1
        #move once for every stepSize
        for i in range(0, self.stepSize):
            
            #one for each direction
            direct = np.random.randint(0,8)
            
            # if the direction is 1,0,7 move x by +1
            if ((direct==0) or (direct==1) or (direct==7)):
                self.location[0] = self.location[0] + 1
                
            # if the direction is 1,2,3 move y by +1
            if ((direct==1) or (direct==2) or (direct==3)):
                self.location[1] = self.location[1] + 1
                
            # if the direction is 3,4,5 move x by -1
            if ((direct==3) or (direct==4) or (direct==5)):
                self.location[0] = self.location[0] - 1
                
            # if the direction is 5,6,7 move y by -1
            if ((direct==5) or (direct==6) or (direct==7)):
                self.location[1] = self.location[1] - 1
            
    # check if location needs to wrap
    def locationCheck(self):
        
        for i in range(0,2):
            if (self.location[i] > self.mapSize):
                self.location[i] = self.location[i] - self.mapSize
            elif (self.location[i] < 0):
                self.location[i] = self.mapSize - self.location[i]
    


In [2]:
class Food:
    location = []
    eaten = False
    mapSize = 0
    
    def __init__(self, mapSize, location = None):
        
        if location == None:
            location = [np.random.randint(0, mapSize), np.random.randint(0, mapSize)]
        self.location = location
        
        self.mapSize = mapSize

In [3]:
class Rabbit(Animal):
    
    probRepro = 0.5
    litter = 1                                     # Edit this value?
    species = 'Rabbit'
    eatMush = False
    
    def reproduced(rabbit2):
        self.hunger = self.hunger * 1.25           # Edit this value?
        rabbit2.hunger = rabbit2.hunger * 1.25     # Edit this value?
        
    def interactRabbit(rabbit, animalArray):
        together = False
        if not rabbit.beStill:
            together = vicinityCheck(self, rabbit)
        if together:
            if ((np.rand.ranf(0, 1) < self.probRepro)):
                for i in range(0, self.litter):
                    reproduce(animalArray, self)
                self.reproduced(rabbit)    
        
    def interactMushroom(mushroom):
        together = vicinityCheck(self, mushroom)
        if together:
            mushroom.eaten = True
            self.hunger = self.hunger / 1.25       # Edit this value?

In [4]:
class Fox(Animal):
    
    probRepro = 0.3
    litter = 1                                         # Edit this value?
    species = 'Fox'
    
    def reproduced(fox):
        self.hunger = self.hunger * 1.25               # Edit this value?
        fox.hunger = fox.hunger * 1.25                 # Edit this value?
        
    def interactRabbit(rabbit):
        
        together = False
        if not rabbit.beStill:
            together = vicinityCheck(self, rabbit)
        if (together):
            rabbit.beStill = True
            self.hunger = self.hunger / 1.25 # Edit this value?
        
    def interactFox(fox, animalArray):
        together = False
        if not fox.beStill:
            together = vicinityCheck(self, fox)
        if together:
            if ((np.rand.ranf(0, 1) < self.probRepro)):
                for i in range(0, self.litter):
                    reproduce(animalArray, self)
                self.reproduced(fox)

In [5]:
class Mushroom(Food):
    
    species = 'Mushroom'

In [6]:
def hungerCheck(animalArray):
        
    everyoneDead = True
    for i in range(0, len(animalArray)):
        
        hunger = animalArray.hunger
        maxHunger = animalArray.maxHunger
        
        if hunger > maxHunger:
            animalArray(i).beStill = True
        else:
            everyoneDead = False
            
    return animalArray, everyoneDead

def vicinity(animal1, animal2):
    
    a1X = animal1.location[0]
    a1Y = animal1.location[1]
    a2X = animal2.location[0]
    a2Y = animal2.location[1]
    sense = animal1.sense
    nearby = False
    
    for i in range(-sense, (sense+1)):
        if (a1X == (a2X + i)):
            i = 2
            for j in range(-sense, (sense+1)):
                if (a1Y == (a2Y + j)):
                    nearby = True
                    j = 2
    return nearby

def reproduce(animalArray, animal):
    
    if(animalArray[0].species == 'Rabbit'):
        animalArray.append(Rabbit(animalArray[0].mapsize, animal.location))
        animalArray[len(animalArray)].step_size = 8
        animalArray[len(animalArray)].step()
        animalArray[len(animalArray)].step_size = animal.step_size
        
    elif(animalArray[0].species == "Fox"):
        animalArray.append(Fox(animalArray[0].mapsize, animal.location))
        animalArray[len(animalArray)].step_size = 8
        animalArray[len(animalArray)].step()
        animalArray[len(animalArray)].step_size = animal.step_size
        
        